In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle as p
cv2.__version__

'3.4.4'

In [2]:
#paths 

folder = './models'
model_path = os.path.join(folder,'ssd.caffemodel')
proto_path = os.path.join(folder,'deploy.prototxt')

In [3]:
dnet = cv2.dnn.readNetFromCaffe(proto_path,model_path)

In [4]:
#face net model
enet = cv2.dnn.readNetFromTorch('./models/openface_nn4.small2.v1.t7')

In [5]:

def filter_op(r,conf = 0.9):
    
    
    box = []
    
    #filtering out using confidence levels
    for i in r[0,0,:]:

        if i[2] > conf:
            b = i[3:] * np.array([w,h,w,h])
            box.append(b.astype('int32'))
        
    return box    


In [6]:
def process(res,conf = 0.9,th = 0.1):
    
    
    rect = filter_op(res,conf)
#     print(len(rect))
#     if len(rect)>1:
#         print('perfroming nms')
#         n_rect = nms(rect,th)
#         return [x for x in n_rect if x.all() != 0]
#     else:
    return rect

In [7]:
def get_embedding(img,r):
    
    face = img[r[1]:r[3],r[0]:r[2]]
    print(face.shape)
    if face.shape[0] == 0:
        return np.zeros(128)
    
    
#     print(face.shape)
    face_blob = cv2.dnn.blobFromImage(face,1/255,(96,96),swapRB = True)
#     print(face_blob.shape)
    enet.setInput(face_blob)
    return enet.forward()
    
    

In [8]:
# emb = []
# name = []
# for path in os.listdir('./people'):
    
#     print(path)
#     for img in os.listdir('./people/'+path):
        
        
#         img_path = './people/'+path+'/'+img
#         print(img_path)
#         i = cv2.imread(img_path)
        
        
#         blob_img = cv2.dnn.blobFromImage(i,1.0,size = (300,300),mean = (123.68,116.78,103.94),swapRB = False,crop = False)
#         dnet.setInput(blob_img)
#         result = dnet.forward()
# #         print(result.shape)
#         res = process(result,conf = 0)
#         print('no of detections:',len(res))
# #         plt.figure()
# #         plt.imshow(i[res[0][1]:res[0][3],res[0][0]:res[0][2]])
# #         plt.show()
#         emb.append(get_embedding(i,res[0]))
#         name.append(path)
        
        
# print('no of embeddigs and names',len(emb),len(name))    
# with open('./pickle_files/'+'emb.pickle','wb') as f:
#     p.dump({'emb':emb,'name':name},f)
        
    

In [9]:
#encoding labels 

from sklearn.preprocessing import LabelEncoder

with open('./pickle_files/emb.pickle','rb') as f:
    data = p.load(f)
    
label  = LabelEncoder()
l = label.fit_transform(data['name'])
print(l)
label.inverse_transform(l)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


/home/clive69/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'unknown', 'unknown', 'clive', 'clive', 'clive', 'clive', 'clive',
       'clive', 'clive', 'clive', 'clive', 'clive', 'clive', 'clive',
       'clive', 'clive', 'clive', 'clive', 'clive', 'clive', 'clive',
       'clive'], dtype='<U7')

In [10]:
# train_test_split?
# ytrain

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

x = np.squeeze(np.array(data['emb']))

xtrain,xtest,ytrain,ytest = train_test_split(x,l,test_size = 0,shuffle = True)

print(xtrain.shape,xtest.shape)
clf = SVC(C = 70.0,gamma = 0.001,probability = True)

clf.fit(xtrain,ytrain)
# np.squeeze(np.array(data['emb'])).shape,l.shape

(40, 128) (0, 128)


SVC(C=70.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
import cv2
#geqttiing training images
cam = cv2.VideoCapture(0)
while 1 :

    _,frame = cam.read()
    
    blob_img = cv2.dnn.blobFromImage(frame,1.0,size = (300,300),mean = (123.68,116.78,103.94),swapRB = False,crop = False)
    h,w,_ = frame.shape
    dnet.setInput(blob_img)
    result = dnet.forward()
#     print(result.shape)
    res = process(result,conf = 0.9)
#     print('no of detections:',len(res))
    
    if len(res) > 0:
        vec  = get_embedding(frame,res[0])
    
#         name = decode(vec)

#         name = label.inverse_transform(clf.predict(vec))[0]
        if vec.all() != 0:     
    
            pred = clf.predict_proba(vec)[0]
            j = np.argmax(pred)
#           print(label.classes_[j])
            score = pred[j]
            name =  label.classes_[j]
            if score < 0.5:
                name  = '----'



    else:
        print('no detction')
    
    
    
    for o in res:
        cv2.rectangle(frame,(o[0],o[1]),(o[2],o[3]),(255,0,0),5)
        cv2.putText(frame,name+':'+str(score),(o[0],o[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)

    
    
    cv2.imshow('img',frame)
    

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cam.release()    
cv2.destroyAllWindows()   
        
    
    



(206, 163, 3)
(210, 167, 3)
(210, 167, 3)
(202, 166, 3)
(212, 170, 3)
(208, 165, 3)
(204, 163, 3)
(207, 160, 3)
(206, 166, 3)
(207, 166, 3)
(209, 164, 3)
(209, 163, 3)
(209, 165, 3)
(206, 162, 3)
(201, 158, 3)
(207, 150, 3)
(205, 149, 3)
(201, 147, 3)
(194, 149, 3)
(194, 143, 3)
(195, 145, 3)
(195, 151, 3)
(193, 150, 3)
(190, 147, 3)
(193, 146, 3)
(188, 148, 3)
(190, 152, 3)
(198, 156, 3)
(179, 140, 3)
(177, 149, 3)
(173, 147, 3)
(168, 143, 3)
(170, 142, 3)
(176, 151, 3)
(168, 148, 3)
(177, 148, 3)
(174, 149, 3)
(171, 144, 3)
(173, 144, 3)
(172, 157, 3)
(173, 160, 3)
(170, 153, 3)
(167, 159, 3)
(169, 163, 3)
(166, 154, 3)
(168, 156, 3)
(166, 163, 3)
(168, 160, 3)
(167, 159, 3)
(171, 160, 3)
(167, 161, 3)
(166, 156, 3)
(163, 159, 3)
(167, 155, 3)
(166, 158, 3)
(171, 158, 3)
(167, 155, 3)
(166, 159, 3)
(167, 162, 3)
(168, 161, 3)
(166, 160, 3)
(163, 153, 3)
(173, 156, 3)
(164, 154, 3)
(187, 173, 3)
(188, 155, 3)
(189, 145, 3)
(193, 144, 3)
(185, 138, 3)
(187, 142, 3)
(192, 145, 3)
(193, 

no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no d

no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no detction
no d